In [ ]:
%load_ext cython

In [ ]:
%%cython

# Куда же без Hello World?

print('Hello, World!')

In [ ]:
%%cython -a

import cython

# О типизации, встроенных питонячьих операциях, и -a

def mult(a, b):
    # PyNumber_Multiply
    return a * b

def mult_1(int a, int b):
    # __Pyx_PyInt_From_int
    return a * b

cpdef int mult_2(int a, int b):
    return a * b

import cython

@cython.cdivision
cdef double d_1(double v1, double v2):
    return v1 / v2

def h():
    return d_1(1, 0)


In [ ]:
%%cython

# def vs cdef vs cpdef
# Обратить внимание, что def всегда принимают на вход PyObject* (object)
# Не принимают указатели и любые типы, которые Cython не сможет скастить в PyObject
# cdef / cpdef МОГУТ принимать тип PyObject*, но получают гораздо больший профит от C - типизации

# У def можно указывать входные, но нельзя выходные - всегда возвращается PyObject*

def x(char* a):
    pass

cdef x_1(int* a):
    pass

x(b'1')

In [ ]:
%%cython -a

# Переполнение int и скорость работы
# def - функции всегда работают с объектами, и статическая типизация не дает нам выигрыша
# с int работаем безопасно, но медленно

def py_fact_1(n):
    if n <= 1:
        return 1
    return n * py_fact_1(n - 1)

def py_fact_2(long n):
    if n <= 1:
        return 1
    return n * py_fact_2(n - 1)

cpdef long py_fact_3(long n):
    if n <= 1:
        return 1
    return n * py_fact_3(n - 1)

def py_fact_4(int n):
    if n <= 1:
        return 1
    return n * py_fact_4(n - 1)

print(py_fact_3(21))
print(py_fact_4(21))

In [ ]:
%%cython

# Работа с указателями
# Не стоит указывать одновременно указатели и переменные

cdef int *a, b, c
b = 1

In [ ]:
%%cython

# Работа с указателями
# Так как * уже зарезрвирован под работу с *args и **kwargs, для разыменовывания используется оператор [0]
# Для операции взятия адреса используется &

cdef: 
    int a
    int *b

b = &a
b[0] = 1

print(a)

In [ ]:
%timeit py_fact_3(21)

In [ ]:
%%cython

# Работа со строками
# Если вы не знаете на 100%, что вы делаете - пользуйтесь Python-строками
# Либо CPP-строками
# В большинстве случаев они быстрее аллоцируют память чем *char строки

cpdef test_str(str a):
    cdef Py_UNICODE* b = a
    print(b)
    print(len(b))

cpdef test_str_1(bytes a):
    cdef char* b = a
#     cdef str c = b Nope
    cdef bytes c = b
    print(c)
    cdef str d = c.decode()
    print(d)

In [ ]:
test_str('Привет')
test_str_1('Привет'.encode())

In [ ]:
%%cython -+

from libcpp.string cimport string

cpdef test_str_2(string s):
    print(s)
    
test_str_2(b'123')

In [ ]:
%%cython

# Преобразование словарей / таплов

cdef struct _inner:
    int inner_a

cdef struct nested:
    int outer_a
    _inner inner
    
cpdef dict test_map():
    cdef nested result = {'outer_a': 1, 'inner': {'inner_a': 2}}
    return result

ctypedef struct tpl:
    int a
    int b
    
    
ctypedef union u:
    int a
    float b

print(test_map())
cdef tpl k = tpl(a=1, b=1)
cdef tpl l = tpl(1, 2)
l.a = 2

cdef p = u(a=1.0)
print(p)

In [ ]:
%%cython

# generic-функции (fused-типы, сейчас в стадии тестирования)

ctypedef fused int_or_float:
    int
    float

cpdef incr(int_or_float val):
    return val + 1

print(incr(1))
print(incr(2.0))

In [ ]:
%%cython

cimport cython

# Если функция принимает несколько fused-параметров одного fused-типа, их базовый тип будет приведен

ctypedef fused int_or_float:
    int
    float

cdef add(int_or_float a, int_or_float b):
    print(cython.typeof(a))
    print(cython.typeof(b))
    return a + b

print(add(1, 1.0))

In [ ]:
%%cython

# Эффективные циклы и динамические массивы
# Cython поддерживает обычную нотацию for .. in, но требует явного указания для итерируемой переменной

from libc.stdlib cimport malloc, free
from libc.string cimport memset

import random

cpdef list dynamic_arr(int N):
    cdef list result = []
    # Обратить внимание, что в C каст идет через ()
    cdef long *arr = <long*>malloc(N * sizeof(long))
    
    # Обязательно проверяем, хватило ли нам памяти
    if not arr:
        raise MemoryError
    
    try:
        # Заполняем массив нулями
        memset(arr, 0, N * sizeof(long))

        for i in range(N):
            result.append(arr[i])
        
    finally:
        # После работы с памятью нужно не забывать ее очищать
        free(arr)
    
    return result


cpdef list square(int N):
    cdef:
        long *arr = <long*>malloc(N * sizeof(long))
        int i
    
    try:
        for i in range(N):
            arr[i] = i * i
        return [x for x in arr[:N]]
    finally:
        free(arr)

print(dynamic_arr(10))
print(square(10))
print(python_alloc(10))

In [ ]:
%%cython

from cpython.mem cimport PyMem_Malloc, PyMem_Free

# Но лучше пользоваться питонячьими аллокаторами, 
# которые могут выделять уже имеющуюся память, и более эффективны в большинстве случаев
# PyMem_Malloc возвращает указатель на сырую память void*, его нужно кастить так же, как и malloc

import random

cpdef list python_alloc(int N):
    cdef:
        long *arr = <long*> PyMem_Malloc(N * sizeof(long))
        int i
    
    try:
        for i in range(N):
            arr[i] = random.randint(0, N)
            print(arr[i])
        return [x for x in arr[:N]]
    finally:
        PyMem_Free(arr)
        
print(python_alloc(10))
    

In [ ]:
%%cython

# Работа с буферами пошла из NumPy и синтаксис очень похож

# А еще лучше - пользоваться интерфейсом буферов
cdef int c_array[3]
# Установка элементов массива
c_array[:] = [1, 2, 3]

print(c_array)

# Буфер - memory view
cdef int[:] buf = c_array

print(buf)

# Работа с исходным массивом через memory-view [...]
buf[...] = 1

print(c_array)

# ... или [:]

buf[:] = 2

print(c_array)

# Несколько измерений

cdef int c_array2[3][3]
cdef int[:, :] buf2 = c_array2

buf2[:, 1] = 1
print(c_array2)


# Динамические массивы
from cpython.mem cimport PyMem_Malloc, PyMem_Free

cdef int *c_array3 = <int*>PyMem_Malloc(3 * sizeof(int))
# Не сработает, нужно явно кастить типы
# cdef int[:] buf3 = c_array3
cdef int[::1] buf3 = <int[:3]>c_array3  # ::1 - это C - нотация для работы с массивами.
print(list(buf3)) # Мусорные значения из памяти

In [ ]:
%%cython -+ -a

# -+ или -cplus - генерация C++ кода

from libcpp.vector cimport vector

# Работа с массивами CPP
# Вызов sum() приводит к обратному переводу vector -> PyObject*!
# Типизация list_test убирает конвертацию возвращаемого значения C -> PyObject

cpdef int list_test(list a):
    cdef vector[int] b
    b = a
    
    cdef:
        int i
        int result = 0
    
    for i in range(b.size()):
        result += i
    
    return result

cpdef int list_test_2(vector[int] a):
    return sum(a)


cdef int list_test_3(vector[int] a):
    return sum(a)


def list_test_3_wrap(list a):
    return list_test_3(a)
    
    
print(list_test([1, 2, 3]))
print(list_test_2([1, 2, 3]))
print(list_test_3_wrap([1, 2, 3]))

# Безопасное преобразование типов
# print(list_test_2([1, 2, 3, '3']))

In [ ]:
%%cython -a

# Обработка ошибок

cpdef int divide_ints_1(int i, int j):
    return i // j

cpdef int divide_ints_2(int i, int j) except? -1:
    return i // j

cpdef int strange() except -1:
    # except ? проверяет, установлен ли глобальный флаг обработки ошибки
    # так как значение -1 может быть валидным для функции
    return -1

cpdef void strange_2() except *:
    # Для функций, не возвращающих результат (void) единственный способ узнать, 
    # что произошла ошибка - except * 
    cdef int a = 1
    cdef int b = 0
    cdef double c = a / b

    
# Для обработки исключений CPP используется except +

In [ ]:
# print(divide_ints_1(1, 0))
# print(divide_ints_2(1, 0))
strange_2()

In [ ]:
%%cython

# Еще о безопасных кастах типов

def test_list(a):
    # TypeError / SystemError
    cdef list cast_list = <list?>a
    print(type(a))
    print(type(cast_list))
    cast_list.append(1)
    

def test_list_2(list a not None):
    # not None проверка для def - функций
    # в cdef делаем вручную
    print(len(a))

# И немного наркомании для работы с указателем на сырую память
def print_address(a):
    cdef void *v = <void*>a
    cdef long addr = <long>v
    print("Cython address:", addr)
    print("Python id     :", id(a))


In [ ]:
print_address(1)

In [ ]:
%%cython 
# extensions

cdef class TestObject:
    cdef long a
    cdef public double b
    cdef readonly str c

In [ ]:
k = TestObject(1, 2.0, b'hello', 1, 6, 7)

In [ ]:
k.b = 1.0

In [ ]:
%%cython

from cpython.mem cimport PyMem_Malloc, PyMem_Free

# cinit
# Cython гарантирует, что __cinit__ вызывается ровно один раз

cdef class Matrix:
    cdef:
        unsigned long long nrows, ncols
        double *_matrix
        public double[:, ::1] matrix
    def __cinit__(self, long nr, long nc):
        self.nrows = nr
        self.ncols = nc
        
        self._matrix = <double*>PyMem_Malloc(nr * nc * sizeof(double))
        
        if self._matrix == NULL:
            raise MemoryError()
            
        self.matrix = <double[:self.nrows, :self.ncols]>self._matrix
        self.matrix[...] = 0
        
    def __dealloc__(self):
        if self._matrix != NULL:
            PyMem_Free(self._matrix)
            
    cpdef double calc_sum(self):
        cdef double result
        
        for i in range(self.nrows):
            for j in range(self.ncols):
                result += self.matrix[i, j]
                
        return result
    
    cpdef void set_item(self, long row, long col, double val):
            
        self.matrix[row, col] = val
        
        
def dispatch(Matrix m not None):
    return m.calc_sum()

In [ ]:
a = Matrix(10, 10)
a.set_item(1, 1, 10)
# a.calc_sum()
a.matrix[5][5] = 5
# a.calc_sum()


# Как получить segfault в Cython?
dispatch(None) # segfault

In [12]:
%%cython

# Работаем с inline C - кодом в докстрингах

cdef extern from *:
    """
    static long square(long x) {return x * x;}
    """
    long square(long x)
    

cdef long a = square(55)
print(a)

3025


In [ ]:
%%cython

from libc.stdlib cimport strtol
from libc.string cimport strncpy , strlen

from libc.stdlib cimport malloc
# ( char * destination, const char * source, size_t num );

cdef enum redis_types:
    PLUS = b'+' # string
    MINUS = b'-' # error
    DOLLAR = b'$' # bytestring
    ASTERISK = b'*' # array
    DOTDOT = b':' # integer
    PERCENT = b'%' # dict
    CR = b'\r'

cdef extern from *:
    """
    char * substr(char * s, int x, int y)
    {
    char * ret = malloc(strlen(s) + 1);
    char * p = ret;
    char * q = &s[x];

    while(x  < y)
    {
        *p++ = *q++;
        x ++; 
    }

    *p++ = '\0';

    return ret;
    }
    """
    char * substr(char * s, int x, int y)

cdef long parse_int(char* s):
    cdef:
        long pos = 1
        char* res
        
    while s[pos] != redis_types.CR:
        pos += 1
    
    return strtol(substr(s, 1, pos), NULL, 10)

cdef char * parse_bytes(char* s):
    return substr(s, 1, strlen(s) - 1)

cpdef parse(char* s):
    cdef long pos = 0
    if s[pos] == redis_types.PLUS:
        return parse_bytes(s)
    elif s[pos] == redis_types.DOTDOT:
        return parse_int(s)

In [ ]:
# %%cython
def parse_resp(f):
    
    b = f.read(1)
    
    if b == b'*':
        data = f.readline()
        l = int(data[:-2])
        arr_res = []
        for i in range(l):
            arr_res.append(parse_resp(f))
        return arr_res
    
    if b == b'+':
        data = f.readline()
        return data[:-2]
    
    elif b == b'-':
        data = f.readline()
        raise Exception(data[:-2])

    elif b == b':':
        data = f.readline()
        return int(data[:-2])
    
    elif b == b'$':
        data = f.readline()
        l = int(data[:-2])
        data = f.read(l)
        f.readline()
        return data

In [ ]:
%%cython

import socket

cdef CRLF = b'\r\n'

from libc.stdlib cimport strtol


cdef long parse_int(char* s):
    return strtol(s, NULL, 10)
            
cpdef parse_resp_c(object f):
    cdef:
        bytes b, data
        cdef int i
        cdef list arr_res
    
    b = f.read(1)
    
    if b == b'*':
        data = f.readline()
        l = parse_int(data[:-2])
        arr_res = []
        for i in range(l):
            arr_res.append(parse_resp_c(f))
        return arr_res
    
    if b == b'+':
        data = f.readline()
        return data[:-2]
    
    elif b == b'-':
        data = f.readline()
        raise Exception(data[:-2])

    elif b == b':':
        data = f.readline()
        return parse_int(data[:-2])
    
    elif b == b'$':
        data = f.readline()
        l = parse_int(data[:-2])
        # Читаем
        data = f.read(l)
        f.readline()
        return data
 

In [ ]:
d = BytesIO(b'*4\r\n:100\r\n+PONG\r\n$6\r\nPONG\r\n\r\n:1000\r\n')
d.seek(0)
%timeit parse_resp_c(d)
d.seek(0)
%timeit parse_resp(d)

In [16]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('localhost', 6379))
sock_f = sock.makefile('rb')
            
sock.send(b'SET B 1\r\n')
print(sock_f.readline())

b'+OK\r\n'
